In [2]:
import os 
os.environ['OMP_NUM_THREADS'] = '1'

In [4]:
from mindquantum import H, Hamiltonian, X, Z, RX, RY, RZ, Circuit
import mindquantum as mq
import numpy as np

In [8]:
H.matrix()

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [9]:
circ = Circuit([H.on(0), X.on(1, 0)])
circ

q0: ──H────●──
           │  
q1: ───────X──

In [10]:
circ += mq.qft([0, 1, 2])
circ

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@──
           │            │          │                            │  
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼──
                                   │               │            │  
q2: ───────────────────────────────●───────────────●───────H────@──

In [12]:
circ += mq.ZZ('theta').on([0, 1])

In [13]:
circ

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(theta)──
           │            │          │                            │        │      
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(theta)──
                                   │               │            │               
q2: ───────────────────────────────●───────────────●───────H────@───────────────

In [25]:
sim = mq.Simulator('projectq', 3)

In [26]:
circ = Circuit()
for i in range(3):
    circ += mq.H(i)

circ += mq.qft(range(3))
sim.apply_circuit(circ)
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [5]:
enconder = Circuit()
enconder += H.on(0)
enconder += RX('alpha0').on(0)
enconder += RY('alpha1').on(0)
enconder += RZ('alpha2').on(0)

In [8]:
enconder.get_qs('projectq', {'alpha0': 0.1, 'alpha1': 0.2, 'alpha2': 0.3}, ket=True)

'(0.6203638230723639-0.1257539726032947j)¦0⟩\n(0.7702994677056821+0.07728774449984671j)¦1⟩'

In [9]:
ansatz = Circuit()
ansatz += RX('theta0').on(0)
ansatz += RY('theta1').on(0)

In [12]:
circ = enconder + ansatz
circ

q0: ──H────RX(alpha0)────RY(alpha1)────RZ(alpha2)────RX(theta0)────RY(theta1)──

In [14]:
sim = mq.Simulator('projectq', circ.n_qubits)
sim

projectq simulator with 1 qubit.
Current quantum state:
1¦0⟩

In [17]:
ham = Hamiltonian(-mq.QubitOperator('Z0'))